In [1]:
import pandas as pd
import numpy as np
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv('../data/raw/nytaxi2022.csv', usecols=[
     "tpep_pickup_datetime", "tpep_dropoff_datetime", "passenger_count", "trip_distance", 
     "RatecodeID", "PULocationID", "DOLocationID", "payment_type", 
     "extra", "total_amount"])
print(df.shape)
# df.head()

(39656098, 10)


In [7]:
#df = df.drop_duplicates()
df = df.dropna()
df = df[df["passenger_count"] > 0]
df = df[df["trip_distance"] > 0]
df = df[df["total_amount"] > 0]
df = df[df['PULocationID'].between(1, 265)] #这两行好像没丢掉数据
df = df[df['DOLocationID'].between(1, 265)]


df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"], format = "%m/%d/%Y %H:%M:%S %p")
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"], format = "%m/%d/%Y %H:%M:%S %p")
trip_dur = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60
df = df[trip_dur > 0]
 

df.shape

(36057725, 10)

In [22]:
#df.head()

In [8]:
#时间
df["pickup_ts"] = df["tpep_pickup_datetime"].astype('int64') // 10**9
df["dropoff_ts"] = df["tpep_dropoff_datetime"].astype('int64') // 10**9
df = df.drop(columns=["tpep_pickup_datetime", "tpep_dropoff_datetime"])

# 类别onehot
# cat_cols = ['RatecodeID','payment_type','PULocationID','DOLocationID']
cat_cols = ['RatecodeID','payment_type']
# df = pd.get_dummies(df, columns=cat_cols, dummy_na=False)
for col in cat_cols:
    dummies = pd.get_dummies(df[col], prefix=col, dummy_na=False)
    df = pd.concat([df, dummies], axis=1)
    df.drop(columns=[col], inplace=True)

num_cols = ['passenger_count','trip_distance','extra','pickup_ts','dropoff_ts', 'PULocationID','DOLocationID']
for col in num_cols:
    col_min = df[col].min()
    col_max = df[col].max()
    df[col] = (df[col] - col_min) / (col_max - col_min)

df.shape
# df.head()

(36057725, 20)

In [9]:
df.head()

,passenger_count,trip_distance,PULocationID,DOLocationID,extra,total_amount,pickup_ts,dropoff_ts,RatecodeID_1.0,RatecodeID_2.0,RatecodeID_3.0,RatecodeID_4.0,RatecodeID_5.0,RatecodeID_6.0,RatecodeID_99.0,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5
0,0.125,0.000021,0.534091,0.890152,0.528104,21.95,0.942081,0.942039,True,False,False,False,False,False,False,True,False,False,False,False
1,0.000,0.000011,0.890152,0.155303,0.475671,13.30,0.942081,0.942038,True,False,False,False,False,False,False,True,False,False,False,False
3,0.000,0.000006,0.428030,0.253788,0.475671,11.80,0.942080,0.942037,True,False,False,False,False,False,False,False,True,False,False,False
6,0.000,0.000027,0.878788,0.325758,0.475671,26.00,0.942080,0.942037,True,False,False,False,False,False,False,True,False,False,False,False
7,0.000,0.000011,0.897727,0.571970,0.475671,12.80,0.942079,0.942036,True,False,False,False,False,False,False,False,True,False,False,False


In [10]:
cols = [c for c in df.columns if c != "total_amount"] + ["total_amount"]
df = df[cols]

In [11]:
df.head()

,passenger_count,trip_distance,PULocationID,DOLocationID,extra,pickup_ts,dropoff_ts,RatecodeID_1.0,RatecodeID_2.0,RatecodeID_3.0,RatecodeID_4.0,RatecodeID_5.0,RatecodeID_6.0,RatecodeID_99.0,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5,total_amount
0,0.125,0.000021,0.534091,0.890152,0.528104,0.942081,0.942039,True,False,False,False,False,False,False,True,False,False,False,False,21.95
1,0.000,0.000011,0.890152,0.155303,0.475671,0.942081,0.942038,True,False,False,False,False,False,False,True,False,False,False,False,13.30
3,0.000,0.000006,0.428030,0.253788,0.475671,0.942080,0.942037,True,False,False,False,False,False,False,False,True,False,False,False,11.80
6,0.000,0.000027,0.878788,0.325758,0.475671,0.942080,0.942037,True,False,False,False,False,False,False,True,False,False,False,False,26.00
7,0.000,0.000011,0.897727,0.571970,0.475671,0.942079,0.942036,True,False,False,False,False,False,False,False,True,False,False,False,12.80


In [12]:
import os

os.makedirs("../data/processed", exist_ok=True)

# 保存为 parquet
df.to_parquet("../data/processed/dataall_normalized.parquet", index=False)
print("Saved to ../data/processed/dataall_normalized.parquet")



Saved to ../data/processed/dataall_normalized.parquet


In [13]:
import pyarrow.parquet as pq

# 读 parquet 文件的元信息
parquet_file = pq.ParquetFile("../data/processed/dataall_normalized.parquet")

print("Number of row groups:", parquet_file.num_row_groups)

# 查看每个 row group 的大小和行数
for i in range(parquet_file.num_row_groups):
    rg_meta = parquet_file.metadata.row_group(i)
    print(f"Row Group {i}: {rg_meta.num_rows} rows, {rg_meta.total_byte_size/1024/1024:.2f} MB")


Number of row groups: 35
Row Group 0: 1048576 rows, 23.08 MB
Row Group 1: 1048576 rows, 22.99 MB
Row Group 2: 1048576 rows, 22.82 MB
Row Group 3: 1048576 rows, 21.96 MB
Row Group 4: 1048576 rows, 22.42 MB
Row Group 5: 1048576 rows, 22.03 MB
Row Group 6: 1048576 rows, 22.38 MB
Row Group 7: 1048576 rows, 22.06 MB
Row Group 8: 1048576 rows, 22.04 MB
Row Group 9: 1048576 rows, 21.69 MB
Row Group 10: 1048576 rows, 22.34 MB
Row Group 11: 1048576 rows, 21.78 MB
Row Group 12: 1048576 rows, 22.28 MB
Row Group 13: 1048576 rows, 21.85 MB
Row Group 14: 1048576 rows, 22.00 MB
Row Group 15: 1048576 rows, 21.98 MB
Row Group 16: 1048576 rows, 22.49 MB
Row Group 17: 1048576 rows, 22.16 MB
Row Group 18: 1048576 rows, 22.19 MB
Row Group 19: 1048576 rows, 22.49 MB
Row Group 20: 1048576 rows, 22.35 MB
Row Group 21: 1048576 rows, 22.74 MB
Row Group 22: 1048576 rows, 22.28 MB
Row Group 23: 1048576 rows, 22.70 MB
Row Group 24: 1048576 rows, 22.13 MB
Row Group 25: 1048576 rows, 22.27 MB
Row Group 26: 1048576 r

In [14]:
parquet_file.read_row_group(1).to_pandas().head()

,passenger_count,trip_distance,PULocationID,DOLocationID,extra,pickup_ts,dropoff_ts,RatecodeID_1.0,RatecodeID_2.0,RatecodeID_3.0,RatecodeID_4.0,RatecodeID_5.0,RatecodeID_6.0,RatecodeID_99.0,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5,total_amount
0,0.000,0.000004,0.617424,0.253788,0.528104,0.943790,0.943747,True,False,False,False,False,False,False,True,False,False,False,False,10.30
1,0.000,0.000019,0.337121,0.606061,0.528104,0.943791,0.943749,True,False,False,False,False,False,False,True,False,False,False,False,23.75
2,0.125,0.000004,0.890152,0.901515,0.528104,0.943789,0.943745,True,False,False,False,False,False,False,True,False,False,False,False,9.55
3,0.125,0.000035,0.534091,0.242424,0.528104,0.943791,0.943750,True,False,False,False,False,False,False,True,False,False,False,False,33.35
4,0.000,0.000005,0.939394,0.871212,0.475671,0.943789,0.943746,True,False,False,False,False,False,False,True,False,False,False,False,10.04


In [15]:
df = pd.read_parquet("../data/processed/dataall_normalized.parquet")  
print(df["total_amount"].min())  
print(df["total_amount"].max())  
print(df["total_amount"].mean())  
print(df["total_amount"].std())  

0.04
401095.62
21.332027053564808
95.72304659421266


In [ ]:
# # 时间转 Unix 秒
# df["pickup_ts"] = df["tpep_pickup_datetime"].astype('int64') // 10**9
# df["dropoff_ts"] = df["tpep_dropoff_datetime"].astype('int64') // 10**9
# df = df.drop(columns=["tpep_pickup_datetime", "tpep_dropoff_datetime"])

# # 类别onehot
# cat_cols = ['RatecodeID','payment_type','PULocationID','DOLocationID']
# df = pd.get_dummies(df, columns=cat_cols, dummy_na=False)

# # 数值归一化
# num_cols = ['passenger_count','trip_distance','extra','pickup_ts','dropoff_ts']
# scaler = MinMaxScaler()
# df[num_cols] = scaler.fit_transform(df[num_cols])

# df.head()

: 